In [36]:
import numpy as np
import scipy.io as io
import os

import keras
from keras.models import Model
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers import Input, Flatten
import keras.backend as K
from keras.constraints import max_norm
from keras.regularizers import l1_l2
from keras.layers.core import Dense, Activation, Dropout, Permute, Reshape
from keras.callbacks import Callback, EarlyStopping
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3636180585040203710
]


In [6]:
K.set_image_data_format('channels_first')

def square(x):
    return K.square(x)

def log(x):
    return K.log(K.clip(x, min_value = 1e-7, max_value = 10000))   

def safe_log(x):
    return K.log(x + 1e-7)

def process_data(train, label, n_class=2, day_index=1, subject_index=1, shuffle=True):
    train = np.squeeze(train[:,:,np.where(label[:,1]==day_index)])
    train = np.transpose(train, axes=[2,0,1])
    train = train.reshape(train.shape[0], 1, train.shape[1], train.shape[2])
    
    label = np.squeeze(label[np.where(label[:,1]==day_index),0])   
    if n_class == 2:
        label[np.where(label != subject_index)] = subject_index + 1
        label = keras.utils.to_categorical(label-subject_index, n_class)
    else:
        label = keras.utils.to_categorical(label-1, n_class)
        
    if shuffle:
        permutation = np.random.permutation(train.shape[0])
        train = train[permutation,:,:]
        label = label[permutation,:]
        
    return train, label 

    Keras implementation of the Shallow Convolutional Network as described
    in Schirrmeister et. al. (2017), arXiv 1703.0505
    
    Assumes the input is a 2-second EEG signal sampled at 128Hz. Note that in 
    the original paper, they do temporal convolutions of length 25 for EEG
    data sampled at 250Hz. We instead use length 13 since the sampling rate is 
    roughly half of the 250Hz which the paper used. The pool_size and stride
    in later layers is also approximately half of what is used in the paper.
    
                     ours        original paper
    pool_size        1, 35       1, 75
    strides          1, 7        1, 15
    conv filters     1, 13       1, 25

In [7]:
def ShallowConvNet(input_shape, n_class=2, conv1_size=(1, 25), conv2_size=(9, 1), conv1_filter=5, conv2_filter=20):
    dropout_rate = 0.5
    
    # start the model
    input_EEG = Input(input_shape)
    block1       = Conv2D(conv1_filter, conv1_size, kernel_constraint = max_norm(2.))(input_EEG)
    block1       = Conv2D(conv2_filter, conv2_size, use_bias=True, kernel_constraint = max_norm(2.))(block1)
    block1       = BatchNormalization(axis=1, epsilon=1e-05, momentum=0.1)(block1)
    block1       = Activation(square)(block1)
    block1       = AveragePooling2D(pool_size=(1, 30), strides=(1, 10))(block1)
    block1       = Activation(safe_log)(block1)
    block1       = Dropout(dropout_rate)(block1)
    flatten      = Flatten()(block1)
    dense        = Dense(n_class, kernel_constraint = max_norm(0.5))(flatten)
    softmax      = Activation('softmax')(dense)
    
    return Model(inputs=input_EEG, outputs=softmax)

class AccLossHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []
        self.loss = []
    def on_epoch_end(self, batch, logs={}):
        self.acc.append([logs.get('acc'), logs.get('val_acc')])
        self.loss.append([logs.get('loss'), logs.get('val_loss')])

In [8]:
datapath = '/Users/dustintt123/SSVEP_CNN/SSVEP_Data/'
SSVEP_l = io.loadmat(datapath+'SSVEP_l.mat')['SSVEP_l']
SSVEP_h = io.loadmat(datapath+'SSVEP_h.mat')['SSVEP_h']
SSVEP_label = io.loadmat(datapath+'SSVEP_label.mat')['label'] # i_subj, i_session, i_stim, i_trial

# binary one-vs-all classification
batch_size = 100
n_epoch = 150
isEarlyStopping = True
n_patience = 10

history_list = []
yhat_list = []
score_list = []

In [13]:
for i in range(8):
    subject = i+1

    x_train, y_train = process_data(SSVEP_l, SSVEP_label, n_class=2, day_index=1, subject_index=subject)
    x_test, y_test = process_data(SSVEP_l, SSVEP_label, n_class=2, day_index=2, subject_index=subject)
    n_trial, n_height, n_channel, n_timestamp = x_train.shape

    # Setup model
    model = ShallowConvNet(x_train.shape[1:])
    model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.adam(),
              metrics=['accuracy'])

    # Setup callbacks
    history = AccLossHistory()

    if isEarlyStopping:
        earlyStopping = EarlyStopping(monitor='val_acc', min_delta=0.001, patience=n_patience, verbose=1, mode='auto')
        callbacks = [history, earlyStopping]
    else:
        callbacks = [history]

    # Model fit
    fit_hist = model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=n_epoch,
          verbose=1,
          validation_split=0.25,
          callbacks=callbacks,
          shuffle=True)

    # Record metrics
    y_hat = model.predict_on_batch(x_test)
    score = model.evaluate(x_test, y_test, verbose = 1)
    history_list.append(history)
    score_list.append(score)
    yhat_list.append(y_hat)
    
    #Save model
    docs_dir=os.path.expanduser('~/SSVEP_CNN/Results/Models/')
    model.save_weights(os.path.join(docs_dir, 'model_%d'%subject))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 1, 9, 247)         0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 5, 9, 223)         130       
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 20, 1, 223)        920       
_________________________________________________________________
batch_normalization_9 (Batch (None, 20, 1, 223)        80        
_________________________________________________________________
activation_25 (Activation)   (None, 20, 1, 223)        0         
_________________________________________________________________
average_pooling2d_9 (Average (None, 20, 1, 20)         0         
_________________________________________________________________
activation_26 (Activation)   (None, 20, 1, 20)         0         
__________

Exception ignored in: 'zmq.backend.cython.message.Frame.__dealloc__'
Traceback (most recent call last):
  File "zmq/backend/cython/checkrc.pxd", line 12, in zmq.backend.cython.checkrc._check_rc (zmq/backend/cython/message.c:4294)
KeyboardInterrupt


Train on 1440 samples, validate on 480 samples
Epoch 1/150


KeyboardInterrupt: 

In [31]:
x4 = len(conv1_size_list)
y4 = len(conv2_size_list)
test_acc4 = np.empty((x4, y4))
test_loss4 = np.empty((x4, y4))
val_acc4 = np.empty((x4, y4))
val_loss4 = np.empty((x4, y4))

for i in range(x4):
    for j in range(y4):
        test_acc4[i][j] = score_list4[i][j][-1]
        test_loss4[i][j] = score_list4[i][j][0]
        val_acc4[i][j] = history_list4[i][j].acc[-1][-1]
        val_loss4[i][j] = history_list4[i][j].loss[-1][0]
        
scores4 = np.array(score_list4)
histories4 = np.array(history_list4)


NameError: name 'conv1_size_list' is not defined

In [ ]:
docs_dir=os.path.expanduser('~/SSVEP_CNN/Results')
np.save(os.path.join(docs_dir, 'ConvSize_3x1_test_acc'), test_acc4)
np.save(os.path.join(docs_dir, 'ConvSize_3x1_val_acc'), val_acc4)
np.save(os.path.join(docs_dir, 'ConvSize_3x1_val_loss'), val_loss4)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

In [ ]:
def heatmap(data, row_labels, col_labels, ax=None,
            cbar_kw={}, cbarlabel="", **kwargs):
    """
    Create a heatmap from a numpy array and two lists of labels.

    Arguments:
        data       : A 2D numpy array of shape (N,M)
        row_labels : A list or array of length N with the labels
                     for the rows
        col_labels : A list or array of length M with the labels
                     for the columns
    Optional arguments:
        ax         : A matplotlib.axes.Axes instance to which the heatmap
                     is plotted. If not provided, use current axes or
                     create a new one.
        cbar_kw    : A dictionary with arguments to
                     :meth:`matplotlib.Figure.colorbar`.
        cbarlabel  : The label for the colorbar
    All other arguments are directly passed on to the imshow call.
    """

    if not ax:
        ax = plt.gca()

    # Plot the heatmap
    im = ax.imshow(data, **kwargs)

    # Create colorbar
    cbar = ax.figure.colorbar(im, ax=ax, **cbar_kw)
    cbar.ax.set_ylabel(cbarlabel, rotation=-90, va="bottom")

    # We want to show all ticks...
    ax.set_xticks(np.arange(data.shape[1]))
    ax.set_yticks(np.arange(data.shape[0]))
    # ... and label them with the respective list entries.
    ax.set_xticklabels(col_labels)
    ax.set_yticklabels(row_labels)

    # Let the horizontal axes labeling appear on top.
    ax.tick_params(top=False, bottom=True,
                   labeltop=False, labelbottom=True)

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=0, ha="center",
             rotation_mode="anchor")

    # Turn spines off and create white grid.
    for edge, spine in ax.spines.items():
        spine.set_visible(False)

    ax.set_xticks(np.arange(data.shape[1]+1)-.5, minor=True)
    ax.set_yticks(np.arange(data.shape[0]+1)-.5, minor=True)
    ax.grid(which="minor", color="w", linestyle='-', linewidth=3)
    ax.tick_params(which="minor", bottom=False, left=False)

    return im, cbar


def annotate_heatmap(im, data=None, valfmt="{x:.2f}",
                     textcolors=["black", "white"],
                     threshold=None, **textkw):
    """
    A function to annotate a heatmap.

    Arguments:
        im         : The AxesImage to be labeled.
    Optional arguments:
        data       : Data used to annotate. If None, the image's data is used.
        valfmt     : The format of the annotations inside the heatmap.
                     This should either use the string format method, e.g.
                     "$ {x:.2f}", or be a :class:`matplotlib.ticker.Formatter`.
        textcolors : A list or array of two color specifications. The first is
                     used for values below a threshold, the second for those
                     above.
        threshold  : Value in data units according to which the colors from
                     textcolors are applied. If None (the default) uses the
                     middle of the colormap as separation.

    Further arguments are passed on to the created text labels.
    """

    if not isinstance(data, (list, np.ndarray)):
        data = im.get_array()

    # Normalize the threshold to the images color range.
    if threshold is not None:
        threshold = im.norm(threshold)
    else:
        threshold = im.norm(data.max())/2.

    # Set default alignment to center, but allow it to be
    # overwritten by textkw.
    kw = dict(horizontalalignment="center",
              verticalalignment="center")
    kw.update(textkw)

    # Get the formatter in case a string is supplied
    if isinstance(valfmt, str):
        valfmt = matplotlib.ticker.StrMethodFormatter(valfmt)

    # Loop over the data and create a `Text` for each "pixel".
    # Change the text's color depending on the data.
    texts = []
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            kw.update(color=textcolors[im.norm(data[i, j]) > threshold])
            text = im.axes.text(j, i, valfmt(data[i, j], None), **kw)
            texts.append(text)

    return texts

In [ ]:
'''PLOTING MODULE'''
fig = plt.figure(figsize=(13,5))

val_plot = fig.add_subplot(121)
# val_plot.imshow(val_acc, interpolation='nearest')
# plt.legend(loc=4, borderaxespad=0.7)
# plt.title('Validation Accuracy Graph')
# plt.ylabel('Conv1 Size')
# plt.xlabel('Conv2 Size')
# plt.yticks(range(len(conv1_size_list)), conv1_size_list)
# plt.xticks(range(len(conv2_size_list)), conv2_size_list)

val_im, cbar = heatmap(val_acc4, conv1_size_list, conv2_size_list, ax=val_plot,
                   cmap='magma_r', cbarlabel="Accuracy")
texts = annotate_heatmap(val_im, valfmt="{x:.3f}")

plt.title('Validation Accuracy Graph')
plt.ylabel('Conv1 Size')
plt.xlabel('Conv2 Size')

test_plot = fig.add_subplot(122)
# test_plot.imshow(test_acc, interpolation='nearest')
# # plt.legend(loc=1, borderaxespad=0.7)
# plt.title('Test Accuracy Graph')
# plt.ylabel('Conv1 Size')
# plt.xlabel('Conv2 Size')
# plt.yticks(range(len(conv1_size_list)), conv1_size_list)
# plt.xticks(range(len(conv2_size_list)), conv2_size_list)

test_im, cbar = heatmap(test_acc4, conv1_size_list, conv2_size_list, ax=test_plot,
                   cmap='magma_r', cbarlabel="Accuracy")
texts = annotate_heatmap(test_im, valfmt="{x:.3f}")

plt.title('Test Accuracy Graph')
plt.ylabel('Conv1 Size')
plt.xlabel('Conv2 Size')

In [ ]:
'''
So far, the best convolutional layer configuration is
Conv1: No. of filters = 5, Filter Size = (1, 25)
Conv2: No. of filters = 10, Filter Size = (9, 1)
'''

In [ ]:
fig = plt.figure(figsize=(15,5))
acc_plot = fig.add_subplot(121)
for i in range(len(conv1_size_list)):
    his = history_list[i]
    acc_plot.plot(np.array(his.acc)[:,1], label='Conv1 Size {}'.format(conv1_size_list[i]))
plt.legend(loc=4, borderaxespad=0.7)
plt.title('Validation Accuracy Graph')
plt.ylabel('Accuracy')
plt.xlabel('Number of Epochs')

loss_plot = fig.add_subplot(122)
for i in range(len(conv1_size_list)):
    his = history_list[i]
    loss_plot.plot(np.array(his.loss)[:,1], label='Conv1 Size {}'.format(conv1_size_list[i]))
plt.legend(loc=1, borderaxespad=0.7)
plt.title('Validation Loss Graph')
plt.ylabel('Loss')
plt.xlabel('Number of Epochs')


In [ ]:
scores = np.array(score_list)
t = range(len(conv1_size_list))

fig, acc_axis = plt.subplots()

color = 'tab:red'
acc_axis.set_ylabel('Accuracy', color=color)
acc_axis.plot(t, scores[:,1], 'x--', color=color)
acc_axis.tick_params(axis='y', labelcolor=color)

loss_axis = acc_axis.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'
loss_axis.set_ylabel('Loss', color=color)  # we already handled the x-label with ax1
loss_axis.plot(t, scores[:,0], 'o--', color=color)
loss_axis.tick_params(axis='y', labelcolor=color)

plt.title('Test Result')
plt.xlabel('Conv1 Size')
plt.xticks(t, conv1_size_list)
fig.tight_layout()  # otherwise the right y-label is slightly clipped
plt.show()